# Tests

In [ ]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

# from collections import defaultdict
# import copy
# import random
# import os
# import shutil
# from urllib.request import urlretrieve

# import albumentations as A
# from albumentations.pytorch import ToTensorV2
# import cv2
# import matplotlib.pyplot as plt
# from tqdm import tqdm
# import torch
# import torch.backends.cudnn as cudnn
# import torch.nn as nn
# import torch.optim
# from torch.utils.data import Dataset, DataLoader
# from torch.optim import lr_scheduler
# import torchvision.models as models
# from torchvision.transforms import ToTensor
# import torch.nn.functional as F

# cudnn.benchmark = True

In [ ]:
# import torchvision.models as models
# import torch.nn as nn

# efficient_net = models.efficientnet_b4(pretrained=True)
# for params in efficient_net.parameters():
#     params.requires_grad = True

# n_features = efficient_net.classifier[1].in_features
# efficient_net.classifier = nn.Sequential(
#   nn.Linear(in_features=n_features, out_features=1024),
#   nn.ReLU(),
#   nn.Linear(in_features=1024, out_features=512),
#   nn.ReLU(),
#   nn.Dropout(p=0.6), 
    
#   nn.Linear(in_features=512 , out_features=7),
#   nn.LogSoftmax(dim=1)  
# )

In [ ]:
# import torchvision.models as models
# import torch.nn as nn

In [ ]:
# eff_netv2 = models.efficientnet_v2_m(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_V2_M_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_V2_M_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
# list(eff_netv2.children())[-2:-1]

[AdaptiveAvgPool2d(output_size=1)]

In [ ]:
# !pip install torchsummary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# from torchsummary import summary

# summary(eff_netv2, (3, 100, 100))

In [ ]:
# import pickle
# from torch.autograd import Variable


# class Model(nn.Module):
    
#     def __init__(self, pretrained=True, num_classes=7):
#         super(Model, self).__init__()
#         efficientnet_v2_m = models.efficientnet_v2_m(pretrained=pretrained)
        
#         self.features = nn.Sequential(*list(efficientnet_v2_m.children())[:-2])  
#         self.features2 = nn.Sequential(*list(efficientnet_v2_m.children())[-2:-1])  
#         self.fc = nn.Linear(1280, 7)  
        
        
#     def forward(self, x):        
#         x = self.features(x)
#         #### 1, 1280, 4, 4
#         feature = self.features2(x)
#         #### 1, 1280, 1, 1
        
#         feature = feature.view(feature.size(0), -1)
#         output = self.fc(feature)
         
#         params = list(self.parameters())
#         fc_weights = params[-2].data
#         fc_weights = fc_weights.view(1, 7, 1280, 1, 1)
#         fc_weights = Variable(fc_weights, requires_grad = False)

#         # attention
#         feat = x.unsqueeze(1) # N * 1 * C * H * W
#         hm = feat * fc_weights
#         hm = hm.sum(2) # N * self.num_labels * H * W

#         return output, hm

In [ ]:
# model = Model()
# summary(model, (3, 100, 100))

# Add Data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

raf_path = '/content/drive/MyDrive/RAF-DB/aligned.zip'

!mkdir -p raf-basic
!mkdir -p raf-basic/Image

!unzip $raf_path -d /content/raf-basic/Image

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: /content/raf-basic/Image/aligned/train_07272_aligned.jpg  
  inflating: /content/raf-basic/Image/aligned/train_07273_aligned.jpg  
  inflating: /content/raf-basic/Image/aligned/train_07274_aligned.jpg  
  inflating: /content/raf-basic/Image/aligned/train_07275_aligned.jpg  
  inflating: /content/raf-basic/Image/aligned/train_07276_aligned.jpg  
  inflating: /content/raf-basic/Image/aligned/train_07277_aligned.jpg  
  inflating: /content/raf-basic/Image/aligned/train_07278_aligned.jpg  
  inflating: /content/raf-basic/Image/aligned/train_07279_aligned.jpg  
  inflating: /content/raf-basic/Image/aligned/train_07280_aligned.jpg  
  inflating: /content/raf-basic/Image/aligned/train_07281_aligned.jpg  
  inflating: /content/raf-basic/Image/aligned/train_07282_aligned.jpg  
  inflating: /content/raf-basic/Image/aligned/train_07283_aligned.jpg  
  inflating: /content/raf-basic/Image/aligned/train_07284_aligned.jp

In [3]:
!mkdir -p raf-basic/EmoLabel
!cp /content/drive/MyDrive/RAF-DB/list_patition_label.txt /content/raf-basic/EmoLabel/list_patition_label.txt

# Imports

In [4]:
import os
import cv2
import csv
import math
import random
import numpy as np
import pandas as pd
import argparse
import pickle

import torch
import torch.nn as nn
from torchvision import transforms
import torchvision.models as models
import torch.utils.data as data
import torch.nn.functional as F

# Utils

In [5]:
import torch
import cv2
import numpy as np
import random


def add_g(image_array, mean=0.0, var=30):
    std = var ** 0.5
    image_add = image_array + np.random.normal(mean, std, image_array.shape)
    image_add = np.clip(image_add, 0, 255).astype(np.uint8)
    return image_add

def flip_image(image_array):
    return cv2.flip(image_array, 1)

def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    
def generate_flip_grid(w, h, device):
    # used to flip attention maps
    x_ = torch.arange(w).view(1, -1).expand(h, -1)
    y_ = torch.arange(h).view(-1, 1).expand(-1, w)
    grid = torch.stack([x_, y_], dim=0).float().to(device)
    grid = grid.unsqueeze(0).expand(1, -1, -1, -1)
    grid[:, 0, :, :] = 2 * grid[:, 0, :, :] / (w - 1) - 1
    grid[:, 1, :, :] = 2 * grid[:, 1, :, :] / (h - 1) - 1
    grid[:, 0, :, :] = -grid[:, 0, :, :]
    return grid

# Dataset

In [6]:
# -*- coding: utf-8 -*-
import os
import cv2
import torch.utils.data as data
import pandas as pd
import random
from torchvision import transforms

class RafDataset(data.Dataset):
    def __init__(self, args, phase, basic_aug=True, transform=None):
        self.raf_path = args.raf_path
        self.phase = phase
        self.basic_aug = basic_aug
        self.transform = transform
        df = pd.read_csv(os.path.join(self.raf_path, 'EmoLabel', args.label_path), sep=' ', header=None)
        
        name_c = 0
        label_c = 1
        if phase == 'train':
            dataset = df[df[name_c].str.startswith('train')]
        else:
            df = pd.read_csv(os.path.join(self.raf_path, 'EmoLabel/list_patition_label.txt'), sep=' ', header=None)
            dataset = df[df[name_c].str.startswith('test')]
            
        self.label = dataset.iloc[:, label_c].values - 1
        images_names = dataset.iloc[:, name_c].values
        self.aug_func = [flip_image, add_g]
        self.file_paths = []
        self.clean = (args.label_path == 'list_patition_label.txt')
        
        for f in images_names:
            f = f.split(".")[0]
            f += '_aligned.jpg'
            file_name = os.path.join(self.raf_path, 'Image/aligned', f)
            self.file_paths.append(file_name)


    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        label = self.label[idx]
        image = cv2.imread(self.file_paths[idx])
            
        image = image[:, :, ::-1]
        
        
        if not self.clean:    
            image1 = image
            image1 = self.aug_func[0](image)
            image1 = self.transform(image1)

        if self.phase == 'train':
            if self.basic_aug and random.uniform(0, 1) > 0.5:
                image = self.aug_func[1](image)

        if self.transform is not None:
            image = self.transform(image)
        
        if self.clean:
            image1 = transforms.RandomHorizontalFlip(p=1)(image)

        return image, label, idx, image1

# Loss

In [7]:
import torch.nn.functional as F
from torch.autograd import Variable

def ACLoss(att_map1, att_map2, grid_l, output):
    flip_grid_large = grid_l.expand(output.size(0), -1, -1, -1)
    flip_grid_large = Variable(flip_grid_large, requires_grad = False)
    flip_grid_large = flip_grid_large.permute(0, 2, 3, 1)
    att_map2_flip = F.grid_sample(att_map2, flip_grid_large, mode = 'bilinear', padding_mode = 'border', align_corners=True)
    flip_loss_l = F.mse_loss(att_map1, att_map2_flip)
    return flip_loss_l
    

# Model

In [8]:
import pickle
from torch.autograd import Variable


class Model(nn.Module):
    
    def __init__(self, args=None, pretrained=True, num_classes=7):
        super(Model, self).__init__()
        efficientnet_v2_m = models.efficientnet_v2_m(pretrained=pretrained)
        
        self.features = nn.Sequential(*list(efficientnet_v2_m.children())[:-2])  
        self.features2 = nn.Sequential(*list(efficientnet_v2_m.children())[-2:-1])  
        self.fc = nn.Linear(1280, 7)  
        
        
    def forward(self, x):        
        x = self.features(x)
        #### 1, 1280, 4, 4
        feature = self.features2(x)
        #### 1, 1280, 1, 1
        
        feature = feature.view(feature.size(0), -1)
        output = self.fc(feature)
         
        params = list(self.parameters())
        fc_weights = params[-2].data
        fc_weights = fc_weights.view(1, 7, 1280, 1, 1)
        fc_weights = Variable(fc_weights, requires_grad = False)

        # attention
        feat = x.unsqueeze(1) # N * 1 * C * H * W
        hm = feat * fc_weights
        hm = hm.sum(2) # N * self.num_labels * H * W

        return output, hm

In [9]:
# from torchsummary import summary

# model = Model()
# summary(model, (3, 100, 100))

# Main

In [10]:
parser = argparse.ArgumentParser()
parser.add_argument('--raf_path', type=str, default='../raf-basic', help='raf_dataset_path')
parser.add_argument('--label_path', type=str, default='list_patition_label.txt', help='label_path')
parser.add_argument('--workers', type=int, default=4, help='number of workers')
parser.add_argument('--batch_size', type=int, default=32, help='batch_size')
parser.add_argument('--w', type=int, default=7, help='width of the attention map')
parser.add_argument('--h', type=int, default=7, help='height of the attention map')
parser.add_argument('--gpu', type=int, default=0, help='the number of the device')
parser.add_argument('--lam', type=float, default=5, help='kl_lambda')
parser.add_argument('--epochs', type=int, default=60, help='number of epochs')


args = parser.parse_args(
    [
        '--raf_path', '/content/raf-basic', 
        '--epochs', '90',
        '--batch_size', '24',
     ],
)

In [11]:
args.epochs

90

In [13]:
weigths_path = '/content/drive/MyDrive/RAF-DB/efficientnetv2_m_25epoch_8957.pth'

In [ ]:
model = Model(args)
model.load_state_dict(torch.load(weigths_path))


if torch.cuda.is_available():
    device = torch.device('cuda:{}'.format(args.gpu))
else:
    device = torch.device('cpu')
    
model.to(device)

In [15]:
def train(args, model, train_loader, optimizer, scheduler, device):
    running_loss = 0.0
    iter_cnt = 0
    correct_sum = 0
    
    model.to(device)
    model.train()

    total_loss = []
    for batch_i, (imgs1, labels, indexes, imgs2) in enumerate(train_loader):
        imgs1 = imgs1.to(device)
        imgs2 = imgs2.to(device)
        labels = labels.to(device)

        criterion = nn.CrossEntropyLoss(reduction='none')

        output, hm1 = model(imgs1)
        output_flip, hm2 = model(imgs2)
        
        grid_l = generate_flip_grid(args.w, args.h, device)
        

        loss1 = nn.CrossEntropyLoss()(output, labels)
        flip_loss_l = ACLoss(hm1, hm2, grid_l, output)

        loss = loss1 + args.lam * flip_loss_l

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        iter_cnt += 1
        _, predicts = torch.max(output, 1)
        correct_num = torch.eq(predicts, labels).sum()
        correct_sum += correct_num
        running_loss += loss

    scheduler.step()
    running_loss = running_loss / iter_cnt
    acc = correct_sum.float() / float(train_loader.dataset.__len__())
    return acc, running_loss


def test(model, test_loader, device):
    with torch.no_grad():
        model.eval()

        running_loss = 0.0
        iter_cnt = 0
        correct_sum = 0
        data_num = 0

        for batch_i, (imgs1, labels, indexes, imgs2) in enumerate(test_loader):
            imgs1 = imgs1.to(device)
            labels = labels.to(device)

            outputs, _ = model(imgs1)

            loss = nn.CrossEntropyLoss()(outputs, labels)

            iter_cnt += 1
            _, predicts = torch.max(outputs, 1)

            correct_num = torch.eq(predicts, labels).sum()
            correct_sum += correct_num

            running_loss += loss
            data_num += outputs.size(0)

        running_loss = running_loss / iter_cnt
        test_acc = correct_sum.float() / float(data_num)
    return test_acc, running_loss

In [ ]:
setup_seed(0)

train_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225]),
    transforms.RandomErasing(scale=(0.02, 0.25)) ])

eval_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225])])



train_dataset = RafDataset(args, phase='train', transform=train_transforms)
test_dataset = RafDataset(args, phase='test', transform=eval_transforms)



train_loader = torch.utils.data.DataLoader(train_dataset,
                                            batch_size=args.batch_size,
                                            shuffle=True,
                                            num_workers=args.workers,
                                            pin_memory=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=args.batch_size,
                                            shuffle=False,
                                            num_workers=args.workers,
                                            pin_memory=True)





optimizer = torch.optim.Adam(model.parameters() , lr=0.0001, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)



for epoch in range(1, args.epochs + 1):
    train_acc, train_loss = train(args, model, train_loader, optimizer, scheduler, device)
    test_acc, test_loss = test(model, test_loader, device)

    print("Epoch [{}], train_loss: {:.4f}, test_loss: {:.4f}, test_acc: {:.4f}".format(
        epoch, train_loss, test_loss, test_acc))

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch [1], train_loss: 0.3506, test_loss: 0.4549, test_acc: 0.8716
Epoch [2], train_loss: 0.3732, test_loss: 0.4357, test_acc: 0.8732
Epoch [3], train_loss: 0.3448, test_loss: 0.4216, test_acc: 0.8768
Epoch [4], train_loss: 0.3303, test_loss: 0.4219, test_acc: 0.8781
Epoch [5], train_loss: 0.3030, test_loss: 0.4417, test_acc: 0.8670
Epoch [6], train_loss: 0.2812, test_loss: 0.4427, test_acc: 0.8709
Epoch [7], train_loss: 0.2544, test_loss: 0.4062, test_acc: 0.8758
Epoch [8], train_loss: 0.2389, test_loss: 0.4212, test_acc: 0.8807
Epoch [9], train_loss: 0.2324, test_loss: 0.4171, test_acc: 0.8846
Epoch [10], train_loss: 0.1998, test_loss: 0.4337, test_acc: 0.8843
Epoch [11], train_loss: 0.1920, test_loss: 0.4420, test_acc: 0.8781
Epoch [12], train_loss: 0.1812, test_loss: 0.4297, test_acc: 0.8840
Epoch [13], train_loss: 0.1739, test_loss: 0.4025, test_acc: 0.8859
Epoch [14], train_loss: 0.1500, test_loss: 0.4185, test_acc: 0.8846
Epoch [15], train_loss: 0.1415, test_loss: 0.4512, test_a

In [ ]:
while True:
    ...

In [ ]:
torch.save(model.state_dict(), 'efficientnetv2_m_25epoch_8957.pth')

In [ ]:
!cp /content/efficientnetv2_m_25epoch_8957.pth /content/drive/MyDrive/RAF-DB/efficientnetv2_m_25epoch_8957.pth

In [ ]:
with open("efficientnetv2_m_25epoch_8957.pkl", "wb") as f:
    pickle.dump(model, f)

In [ ]:
!cp /content/efficientnetv2_m_25epoch_8957.pkl /content/drive/MyDrive/RAF-DB/efficientnetv2_m_25epoch_8957.pkl